In [ ]:
import pandas as pd
import geopandas as gpd 
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import plotly.express as px

plt.style.use('seaborn-v0_8')  # Estilo moderno
plt.rcParams['font.family'] = 'Arial'  # Fuente profesional

from mypackage import dir

# Environment variables
modality = 'u'
project = 'Ciencia de los datos'
data = dir.make_dir_line(modality, project) 
raw = data('raw')
processed = data('processed')
outputs = data('outputs')

In [ ]:
def cargar_datos(table_name: str) -> pd.DataFrame:
    df = pd.read_parquet(processed / f'{table_name}.parquet.gzip')
    print(f'Loaded table: {table_name}')
    return df

# Función para cargar los datos en la base de datos
def cargar_datos_geo(table_name: str) -> gpd.GeoDataFrame:
    geo = gpd.read_file(processed / f'{table_name}.geojson', encoding='utf-8')
    print(f'Loaded geo table: {table_name}')
    return geo

def create_italy_choropleth(
    df,
    color_column='value',
    location_id='geoid',
    hover_vars=None,  # Dictionary of {column: label} for hover data
    title="Distribución Poblacional en Italia por Grupos Demográficos",
    source_text="Fuente: XYZ",
    center_lat=41.93309,
    center_lon=12.13129,
    zoom=3.8,
    opacity=0.4,
    map_style='carto-positron',
    color_scale=px.colors.sequential.Blues
):
    """Create an elegant choropleth map of Italy with customizable hover data.
    
    Args:
        df: GeoDataFrame with geographical data and values to plot
        color_column: Column name for color values
        location_id: Column name for location IDs (must match geojson properties)
        hover_vars: Dictionary of {column: label} for hover data display
                   Example: {'DEN_PROV': 'Territorio', 'value': 'Unidades'}
        title: Map title
        source_text: Text for data source annotation
        center_lat: Latitude for map center
        center_lon: Longitude for map center
        zoom: Initial zoom level
        opacity: Opacity of choropleth fill
        map_style: Mapbox style specification
        color_scale: Color scale for the choropleth
    
    Returns:
        A Plotly Figure object
    """
    # Set default hover variables if none provided
    if hover_vars is None:
        hover_vars = {
            'DEN_PROV': 'Territorio',
            color_column: 'Valor'
        }
    
    # Prepare hover data and labels
    hover_data = list(hover_vars.keys())
    hover_labels = {col: label for col, label in hover_vars.items()}
    
    fig = px.choropleth_mapbox(
        df,
        geojson=df.__geo_interface__,
        locations=location_id,
        color=color_column,
        hover_data=hover_data,
        labels=hover_labels,
        featureidkey=f'properties.{location_id}',
        center={'lat': center_lat, 'lon': center_lon},
        mapbox_style=map_style,
        zoom=zoom,
        opacity=opacity,
        title=title,
        # color_continuous_scale=color_scale
    )

    # Generate dynamic hover template
    hover_template_parts = [
        f"<b>{label}</b>: %{{customdata[{i}]}}"
        if not isinstance(df[col].iloc[0], (int, np.int64, float))
        else (
            f"<b>{label}</b>: %{{customdata[{i}]:,}}"
            if df[col].iloc[0] > 1
            # else f"<b>{label}</b>: %{{customdata[{i}]:.3f}}"
            else f"<b>{label}</b>: %{{customdata[{i}]:.2%}}"
        )
        for i, (col, label) in enumerate(hover_vars.items())
    ]

    fig.update_traces(
        hovertemplate="<br>".join(hover_template_parts) + "<extra></extra>",
        marker_line_width=0.5,
        marker_line_color='white'
    )

    # Update layout for better appearance
    fig.update_layout(
        margin={'l': 0, 't': 60, 'r': 0, 'b': 40},
        title={
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 16}
        },
        annotations=[dict(
            text=source_text,
            x=0.5,
            y=-0.1,
            showarrow=False,
            xref="paper",
            yref="paper",
            font={'size': 10}
        )],
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial"
        )
    )

    # Update colorbar
    colorbar_title = hover_vars.get(color_column, 'Value')
    fig.update_coloraxes(
        colorbar=dict(
            title=dict(text=colorbar_title, side='right'),
            lenmode="fraction",
            len=0.6,
            yanchor="middle",
            y=0.5,
            x=0,
            thickness=15,
            tickformat=","
        )
    )

    return fig

In [ ]:
provincias = cargar_datos('provincias')
geo_provincias = cargar_datos_geo('provincias')
geo_provincias = pd.merge(geo_provincias, provincias, on=['COD_PROV'])
geo_provincias.head()

In [ ]:
censo = cargar_datos('censo_tranformado')
censo = censo.loc[:,['itter107', 'territory', 'gender', 'value']]
censo = censo.groupby(['itter107', 'territory', 'gender'], observed=False)['value'].sum().reset_index()
censo.head()

In [ ]:
df = geo_provincias.merge(censo, on = ['itter107'])

df['geoid'] = range(1, len(df) + 1)
df['geoid'] = df.index.astype(str)

df.head()

In [ ]:
mapita = create_italy_choropleth(
    df,
    color_column='value',
    hover_vars={"DEN_PROV": 'Territorio',
                'value':'Unidades'},
    # value_label='Unidades',
    # location_id='geoid',
    title="Distribución Poblacional en Italia por Grupos Demográficos",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
# mapita.write_html(output / 'distribucion_poblacional_grupos_italia.html')
mapita

In [ ]:
df = df.pivot(index=['COD_PROV', 'geometry', 'DEN_PROV', 'itter107', 'territory'], 
              columns='gender', values='value').reset_index()

df['poblacion'] = df['females'] + df['males']
df['proportion'] = df['females'] / df['poblacion']

df['value'] = df['poblacion'].copy()

df = gpd.GeoDataFrame(df, geometry='geometry')
df = df.set_crs(epsg=4326)

df['geoid'] = range(1, len(df) + 1)
df['geoid'] = df.index.astype(str)

df.head()

In [ ]:
mapita = create_italy_choropleth(
    df,
    color_column='value',
    hover_vars={"DEN_PROV": 'Territorio', 
                'males':'Hombres',
                'females': 'Mujeres',
                'value':'Unidades', 
                'proportion': 'Proporción'},
    # value_label='Unidades',
    # location_id='geoid',
    title="Distribución Poblacional en Italia por Grupos Demográficos",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
# mapita.write_html(output / 'distribucion_poblacional_grupos_italia.html')
mapita

In [ ]:
# Crear el mapa
fig, ax = plt.subplots(figsize=(20, 15))

# Graficar los polígonosw
df.plot(ax=ax, column='value', legend=True, cmap='coolwarm', edgecolor='black', linewidth=0.5)

# Agregar título
ax.set_title('Distribución Poblacional en Italia', fontsize=15)

# # Guardar el mapa
# plt.savefig('../mapas/mapa_plt.png', dpi=300)

# Mostrar el mapa
plt.show()

## Análisis de Distribución Geográfica

In [ ]:
censo = cargar_datos('censo_tranformado')

censo['grupo_edad'] = 'adultos' 
censo.loc[censo['age'] < 18, 'grupo_edad'] = 'jóvenes'
censo.loc[censo['age'] > 65, 'grupo_edad'] = 'mayores'

censo_total = censo.groupby(['itter107'], observed=False)['value'].sum().reset_index()
censo_total.rename(columns={'value':'value_total'}, inplace=True)


censo = censo.loc[:,['itter107', 'territory', 'grupo_edad', 'value']]
censo = censo.groupby(['itter107', 'territory', 'grupo_edad'], observed=False)['value'].sum().reset_index()

censo = censo.merge(censo_total, on = ['itter107'])
censo['value'] = censo['value']/censo['value_total']

df = geo_provincias.merge(censo, on = ['itter107'])

df['geoid'] = range(1, len(df) + 1)
df['geoid'] = df.index.astype(str)

df_jovenes = df[df['grupo_edad'] == 'jóvenes']
df_adultos = df[df['grupo_edad'] == 'adultos']
df_mayores = df[df['grupo_edad'] == 'mayores']

df_mayores.head()

In [ ]:
mapita = create_italy_choropleth(
    df_jovenes,
    color_column='value',
    hover_vars={"DEN_PROV": 'Territorio',
                'value':'Unidades'},
    # value_label='Unidades',
    # location_id='geoid',
    title="Distribución Poblacional en Italia por Grupos Demográficos - Población jovenes",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
# mapita.write_html(output / 'distribucion_poblacional_grupos_italia_jovenes.html')
mapita

In [ ]:
# Crear la figura con 3 subplots horizontales (1 fila, 3 columnas)
fig, axes = plt.subplots(1, 3, figsize=(30, 10), facecolor='white')
fig.suptitle('Distribución Poblacional en Italia por Grupos de Edad', 
             fontsize=20, y=1.05, fontweight='bold')

# Suponiendo que tienes 3 DataFrames diferentes (df1, df2, df3) para cada mapa
# Si es el mismo DataFrame con diferentes columnas, ajusta según corresponda

# Lista de DataFrames y títulos
dfs = [df_jovenes, df_adultos, df_mayores]
titles = ['Población Joven (15-24 años)', 
          'Población Adulta (25-64 años)', 
          'Población Mayor (65+ años)']
cmap = 'viridis'  # Mejor para datos secuenciales


# Crear cada submapa
for ax, df, title in zip(axes, dfs, titles):
    # Mapa principal
    plot = df.plot(ax=ax, column='value', cmap=cmap, 
                   edgecolor='white', linewidth=0.3)
    
    # Título
    ax.set_title(title, fontsize=16, pad=15, fontweight='semibold')
    
    # Remover ejes
    ax.set_axis_off()
    
    # Barra de color profesional
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    sm = plt.cm.ScalarMappable(cmap=cmap)
    sm._A = []
    fig.colorbar(sm, cax=cax, orientation='vertical')
    cax.set_ylabel('Habitantes', rotation=90, labelpad=15, fontsize=12)

# Ajustes finales
plt.tight_layout(pad=3.0)  # Espaciado entre subplots

# # Añadir anotaciones
# plt.annotate('Fuente: Instituto Nacional de Estadística de Italia | Elaboración propia', 
#              xy=(0.5, 0), 
#              xytext=(0, -30),
#              xycoords='figure points', textcoords='offset points',
#              ha='center', va='top', fontsize=12, color='gray'
#              )

# Guardar en alta calidad
# plt.savefig('mapas_poblacion_italia.png', dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())

# Mostrar el gráfico
plt.show()

Los mapas presentan la distribución geográfica de la población italiana segmentada en tres grupos etarios: jóvenes (15-24 años), adultos (25-64 años) y mayores (65+ años). A pesar de errores tipográficos en los títulos (ej. "Polikación" por "Población"), los datos permiten identificar patrones demográficos clave:

Hallazgos principales:

- Población joven (15-24 años):

Se concentra en áreas urbanas y regiones con universidades (ej. Lombardía, Lacio).
La menor densidad en el sur sugiere migración hacia zonas con mayores oportunidades educativas y laborales.

- Población adulta (25-64 años):

Dominante en regiones económicas clave (Milán, Turín, Veneto), reflejando la fuerza laboral activa.
Correlación con polos industriales y de servicios.

- Población mayor (65+ años):

Mayor presencia en el sur (ej. Sicilia, Cerdeña) y zonas rurales, indicando envejecimiento demográfico y posible éxodo juvenil.

Requiere atención en políticas de salud y pensiones.

<b> Tendencias críticas: </b>

Desbalance norte-sur: El norte muestra mayor dinamismo poblacional (adultos/jóvenes), mientras el sur enfrenta envejecimiento y despoblación.

Urbanización: Las ciudades atraen a jóvenes y adultos, exacerbando desigualdades regionales.

## Visualizar la proporción de personas solteras, casadas o divorciadas por área geográfica.

In [ ]:
censo = cargar_datos('censo_tranformado')
censo = censo[censo['marital_status'].isin(['never married', 'married', 'divorced'])]

censo_total = censo.groupby(['itter107'], observed=False)['value'].sum().reset_index()
censo_total.rename(columns={'value':'value_total'}, inplace=True)


censo = censo.loc[:,['itter107', 'territory', 'marital_status', 'value']]
censo = censo.groupby(['itter107', 'territory', 'marital_status'], observed=False)['value'].sum().reset_index()

censo = censo.merge(censo_total, on = ['itter107'])
censo['value'] = censo['value']/censo['value_total']

df = geo_provincias.merge(censo, on = ['itter107'])

df['geoid'] = range(1, len(df) + 1)
df['geoid'] = df.index.astype(str)

df_never_married = df[df['marital_status'] == 'never married']
df_married = df[df['marital_status'] == 'married']
df_divorced = df[df['marital_status'] == 'divorced']

df_divorced.head()

In [ ]:
mapita = create_italy_choropleth(
    df_never_married,
    color_column='value',
    hover_vars={"DEN_PROV": 'Territorio',
                'value':'Unidades'},
    # value_label='Unidades',
    # location_id='geoid',
    title="Distribución Matrimonios en Italia - Población nunca casada",
    source_text="Fuente: Datos demográficos 2025 | Visualización: Python",
    # center_lat=41.93309,
    # center_lon=12.13129,
    # zoom=3.8,
    # opacity=0.4,
    # map_style='carto-positron'
)
# mapita.write_html(output / 'distribucion_married_italy.html')
mapita

In [ ]:
# Crear la figura con 3 subplots horizontales (1 fila, 3 columnas)
fig, axes = plt.subplots(1, 3, figsize=(30, 10), facecolor='white')
fig.suptitle('Distribución Poblacional en Italia por estado civil', 
             fontsize=20, y=1.05, fontweight='bold')

# Suponiendo que tienes 3 DataFrames diferentes (df1, df2, df3) para cada mapa
# Si es el mismo DataFrame con diferentes columnas, ajusta según corresponda

# Lista de DataFrames y títulos
dfs = [df_never_married, df_married, df_divorced]
titles = ['Población nunca casada', 
          'Población Casada', 
          'Población divorciada']
cmap = 'viridis'  # Mejor para datos secuenciales

# Crear cada submapa
for ax, df, title in zip(axes, dfs, titles):
    # Mapa principal
    plot = df.plot(ax=ax, column='value', cmap=cmap, 
                   edgecolor='white', linewidth=0.3)
    
    # Título
    ax.set_title(title, fontsize=16, pad=15, fontweight='semibold')
    
    # Remover ejes
    ax.set_axis_off()
    
    # Barra de color profesional
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    sm = plt.cm.ScalarMappable(cmap=cmap)
    sm._A = []
    fig.colorbar(sm, cax=cax, orientation='vertical')
    cax.set_ylabel('Habitantes', rotation=90, labelpad=15, fontsize=12)

# Ajustes finales
plt.tight_layout(pad=3.0)  # Espaciado entre subplots

# Mostrar el gráfico
plt.show()